# Bayesian parameter estimation 

Written for CRDM task as part of the IDM dataset collected online with Mturk

In [16]:
# Built-in/Generic Imports
import os,sys
import glob
import time

# Libs
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
import pickle

import logging
logger = logging.getLogger("pymc")
logger.setLevel(logging.ERROR)


In [17]:
# getting the name of the directory
# where the this file is present.
# current = os.path.dirname(os.path.realpath(__file__))
current = os.path.dirname(os.getcwd())
 
# Getting the parent directory name
# where the current directory is present.
# parent = os.path.dirname(os.path.dirname(current))
# parent = current
parent = '/Users/pizarror/IDM'
# print(parent)
#/Users/pizarror/IDM

# adding the parent directory to
# the sys.path.
sys.path.append(parent)

from IDM_model.src import model_functions as mf


In [18]:
def read_load_data(subject='23_IDM_0144',fn='/tmp',alpha0=1.0,cols=[]):
    crdm_df = pd.read_csv(fn)
    task='crdm'
    crdm_df = mf.drop_pract(crdm_df,task=task)
    crdm_df,response_rate = mf.drop_non_responses(crdm_df,task=task,verbose=False) 
    data = mf.get_data(crdm_df,cols,task='crdm')[0]
    data['subject'] = subject
    return data 

def diganostic_plots(trace,experiment='experiment',utility_dir='/tmp/',subject='23_IDM_0001',task='crdm',coords={},var_names=['alpha','beta','gamma'],figsize=(10,10)):

    bh_dir = os.path.join(utility_dir,subject,task,'bh')
    if not os.path.exists(bh_dir):
        os.makedirs(bh_dir)
    print('Saving diagnostic plots to bh_dir : {}'.format(bh_dir))

    title_dict = {'fontsize':15}

    # 2by2 : rows 2 varialbes, cols 2 for distribution and sampled values
    axes = az.plot_trace(trace, var_names=var_names,coords=coords,compact=False)
    for r in range(axes.shape[0]):
        for c in range(axes.shape[1]):
            axes[r,c].set_title('{}: {}'.format(subject,var_names[r]))
    plt.tight_layout()
    fig_fn = os.path.join(bh_dir,'{}_{}_trace_plot.{}.eps'.format(subject,task,experiment))
    plt.savefig(fig_fn,format='eps')
    plt.close()
    
    axes = az.plot_pair(trace,kind='kde', coords=coords,var_names=var_names,marginals=True)
    axes[0,0].set_title(subject,fontdict=title_dict)
    axes[1,0].set_ylabel(var_names[1])
    axes[1,0].set_xlabel(var_names[0])
    plt.tight_layout()
    fig_fn = os.path.join(bh_dir,'{}_{}_bivariate_densities.{}.eps'.format(subject,task,experiment))
    plt.savefig(fig_fn,format='eps')
    plt.close()
    
    axes = az.plot_posterior(trace,var_names=var_names,coords=coords)
    # print(axes.shape)
    for c in range(axes.shape[0]):
        axes[c].set_title('{}: {}'.format(subject,var_names[c]),fontdict=title_dict)
    fig_fn = os.path.join(bh_dir,'{}_{}_posterior.{}.eps'.format(subject,task,experiment))
    plt.savefig(fig_fn,format='eps')
    plt.close()

    fig, axes = plt.subplots(1,len(var_names), figsize=figsize)
    az.plot_rank(trace,var_names=var_names,coords=coords,ax=axes)
    for c in range(axes.shape[0]):
        axes[c].set_title('{}: {}'.format(subject,var_names[c]),fontdict=title_dict)
    fig.tight_layout()
    fig_fn = os.path.join(bh_dir,'{}_{}_rank_plot_bars.{}.eps'.format(subject,task,experiment))
    plt.savefig(fig_fn,format='eps')
    plt.close()

    fig, axes = plt.subplots(1,len(var_names), figsize=figsize)
    axes = az.plot_rank(trace,var_names=var_names, coords=coords,kind="vlines",vlines_kwargs={'lw':0}, marker_vlines_kwargs={'lw':3},ax=axes)
    for c in range(axes.shape[0]):
        axes[c].set_title('{}: {}'.format(subject,var_names[c]),fontdict=title_dict)
    fig.tight_layout()
    fig_fn = os.path.join(bh_dir,'{}_{}_rank_plot_lines.{}.eps'.format(subject,task,experiment))
    plt.savefig(fig_fn,format='eps')
    plt.close()




## Bayesian Hierarchical Model (BHM)

We pooled all data together and ran simple BH model in complete_pool_as_prior.

We are using complete pooling to generate priors for when we implement a higherarchical bayesian model.

### Load data

We will load the data from all participants so we can run the modeling schema

In [19]:
# Takes about 10 seconds

# we will change this when we change utility to 1st level analysis (or split)
split_dir = '/Volumes/UCDN/datasets/IDM/split/'
utility_dir = '/Volumes/UCDN/datasets/IDM/utility/'
save_dir = '/Volumes/UCDN/datasets/IDM/utility/BHM/crdm'
subjs = sorted(glob.glob(os.path.join(split_dir,'23_IDM_*')))
task = 'crdm'
cols = ['crdm_choice','crdm_sure_p','crdm_lott_p','crdm_sure_amt','crdm_lott_amt','crdm_amb_lev']
data = pd.DataFrame(columns=['subject']+cols)

for subj_id,s in enumerate(subjs):
    subject = os.path.basename(s)
    fn  = os.path.join(s,task,'{}_{}.csv'.format(os.path.basename(s),task))
    if os.path.exists(fn):
        subj_data = read_load_data(subject=subject,fn=fn,alpha0=1.0,cols=cols)
        for c in cols:
            subj_data[c] = subj_data[c].astype(float)        
        subj_data['subject_id'] = int(subj_id)
        data = pd.concat([data,subj_data],ignore_index=True)

data.head(10)


,subject,crdm_choice,crdm_sure_p,crdm_lott_p,crdm_sure_amt,crdm_lott_amt,crdm_amb_lev,subject_id
0,23_IDM_0001,1.0,1.0,0.50,5.0,20.0,0.5,0.0
1,23_IDM_0001,1.0,1.0,0.25,5.0,50.0,0.0,0.0
2,23_IDM_0001,0.0,1.0,0.75,5.0,5.0,0.0,0.0
3,23_IDM_0001,0.0,1.0,0.25,5.0,5.0,0.0,0.0
4,23_IDM_0001,0.0,1.0,0.13,5.0,5.0,0.0,0.0
5,23_IDM_0001,1.0,1.0,0.50,5.0,20.0,0.0,0.0
6,23_IDM_0001,1.0,1.0,0.38,5.0,20.0,0.0,0.0
7,23_IDM_0001,1.0,1.0,0.38,5.0,50.0,0.0,0.0
8,23_IDM_0001,0.0,1.0,0.38,5.0,5.0,0.0,0.0
9,23_IDM_0001,1.0,1.0,0.25,5.0,40.0,0.0,0.0


In [20]:

subjects = data['subject'].unique()
# subjects = subjects[:10]
# print(subjects)
data = data.loc[data['subject'].isin(subjects)]


In [21]:

subjects = data['subject'].unique()
nb_subj = subjects.shape[0]
nb_trials = data.shape[0]//nb_subj
subj_id_list = data['subject_id'].to_list()
subj_id = [int(s) for s in subj_id_list]

# subj_id = data['subject'].to_list()

old_id = np.array([ [s]*nb_trials for s in range(nb_subj) ]).flatten()

sure_p = data['crdm_sure_p'].values
lott_p = data['crdm_lott_p'].values
sure_amt = data['crdm_sure_amt'].values
lott_amt = data['crdm_lott_amt'].values
ambig = data['crdm_amb_lev'].values
choices = data['crdm_choice'].values


### Bayesian Hierarchical Model

Developed under parameter receovery


In [22]:
tStep1 = time.time()

# We will fit a model for each subject
with pm.Model() as model_simple:

    # Hyperparameters for alpha, beta
    mu_alpha_hyper = pm.Normal('mu_alpha_hyper',mu=-0.635,sigma=0.05)
    sd_alpha_hyper = pm.Normal('sd_alpha_hyper',mu=0.418,sigma=0.01)

    mu_beta_hyper = pm.Normal('mu_beta_hyper',mu=0.254,sigma=0.05)
    sd_beta_hyper = pm.Normal('sd_beta_hyper',mu=0.575,sigma=0.01)
    
    # mu_gamma_hyper = pm.Normal('mu_gamma_hyper',mu=3.011,sigma=0.05)
    sd_gamma_hyper = pm.Normal('sd_gamma_hyper',mu=2.046,sigma=0.01)

    alpha = pm.LogNormal('alpha',mu=mu_alpha_hyper, sigma=sd_alpha_hyper,shape=np.size(np.unique(subj_id)))
    beta = pm.Normal('beta',mu=mu_beta_hyper,sigma=sd_beta_hyper,shape=np.size(np.unique(subj_id)))
    gamma = pm.HalfNormal('gamma',sigma=sd_gamma_hyper,shape=np.size(np.unique(subj_id)))
    
    prob = pm.Deterministic('prob', 1 / (1 + pm.math.exp(-gamma[subj_id] * 
                                                            ((lott_amt**alpha[subj_id])*(lott_p-beta[subj_id]*ambig/2)
                                                            - (sure_amt**alpha[subj_id])*sure_p)))) 
    
    y_1 = pm.Bernoulli('y_1',p=prob,observed=choices)

    # trace_prior = pm.sample(100, tune=20, cores=2,target_accept=0.95,progressbar=False)
    trace_prior = pm.sample(10000, tune=1000, cores=5,target_accept=0.99,progressbar=False)



# This is how you get a nice array. Note that this returns a pandas DataFrame, not a numpy array. Indexing is totally different.
summary= az.summary(trace_prior,round_to=10)
fn = os.path.join(save_dir,'BHM_model_summary_v002.csv')
print('Saving to : {}'.format(fn))
summary.to_csv(fn)

fn = os.path.join(save_dir,'BHM_model_trace_v002.pkl')
print('Saving to : {}'.format(fn))
with open(fn,'wb') as buff:
    pickle.dump({'trace':trace_prior},buff)
    # pm.save_trace(trace_prior,fn)

print('Time to complete {} aggregate BHM : {} minutes'.format(len(subjects),(time.time() - tStep1)/60.0))


/Users/pizarror/opt/miniconda/envs/idm_jupy/lib/python3.9/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/Users/pizarror/opt/miniconda/envs/idm_jupy/lib/python3.9/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


Saving to : /Volumes/UCDN/datasets/IDM/utility/BHM/crdm/BHM_model_summary_v002.csv
Saving to : /Volumes/UCDN/datasets/IDM/utility/BHM/crdm/BHM_model_trace_v002.pkl
Time to complete 149 aggregate BHM : 24.833476114273072 minutes


### Extract parameter estimates and save

We can incorporate this into the script above

In [23]:
def extract_mean(fn,var_names=['alpha','beta','gamma'],subjects=[]):
    nb_subjects = len(subjects)
    df = pd.read_csv(fn,index_col=0)
    df_bhm = pd.DataFrame([],columns=var_names)
    df_bhm['subject'] = subjects
    for var in var_names:
        ind_list = ['{}[{}]'.format(var,sub_id) for sub_id in range(nb_subjects)]
        df_bhm[var] = df.loc[df.index.isin(ind_list)]['mean'].reset_index(drop=True)
    return df_bhm

experiment='v002'
bhm_dir = '/Volumes/UCDN/datasets/IDM/utility/BHM/'
bhm_fn = os.path.join(bhm_dir,'crdm','BHM_model_summary_{}.csv'.format(experiment))
df_bhm = extract_mean(bhm_fn,var_names=['alpha','beta','gamma'],subjects=subjects)
split_CRDM_fn = os.path.join(bhm_dir,'split_CRDM_BHM.csv')
df_bhm.to_csv(split_CRDM_fn)
df_bhm

,alpha,beta,gamma,subjects
0,1.236226,0.472010,0.825023,23_IDM_0001
1,0.147275,0.751427,5.490793,23_IDM_0002
2,0.708625,0.264056,1.869803,23_IDM_0003
3,0.464319,-0.366157,2.320312,23_IDM_0004
4,0.557125,0.467945,1.720650,23_IDM_0005
...,...,...,...,...
144,0.473570,0.507791,3.720521,23_IDM_0145
145,0.684302,0.006735,1.595158,23_IDM_0146
146,0.263881,0.890332,5.641400,23_IDM_0147
147,0.592807,0.706722,1.005348,23_IDM_0148


### Diagnostic plots

Too many subjects to run `diagnistic_plots()`  but can run them individually
Trace, posterior, rank plots

... need to figure out for each subject how to plot_pair() which plots the bivariate distirbutions


In [24]:
for s in set(subj_id):
    coords={'alpha_dim_0': [s],'beta_dim_0': [s],'gamma_dim_0':[s]}
    diganostic_plots(trace_prior,experiment=experiment,utility_dir=utility_dir,subject=subjects[s],coords=coords,var_names=['alpha','beta','gamma'],figsize=(10,10))


Saving diagnostic plots to bh_dir : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0001/crdm/bh


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


Saving diagnostic plots to bh_dir : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0002/crdm/bh


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


Saving diagnostic plots to bh_dir : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0003/crdm/bh


...

Saving diagnostic plots to bh_dir : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0146/crdm/bh
Saving diagnostic plots to bh_dir : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0147/crdm/bh


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


Saving diagnostic plots to bh_dir : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0148/crdm/bh


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


Saving diagnostic plots to bh_dir : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0149/crdm/bh
